In [1]:
import lxml.etree as et
from glob import glob

doc = "/home/larsm/my_projects/ParlaMint/Data/ParlaMint-NO/ParlaMint-NO_2008-04-08-lower.ana.xml"
doc = "/home/larsm/my_projects/ParlaMint/Data/ParlaMint-NO/ParlaMint-NO_2004-09-30.xml"
file_paths = "/home/larsm/my_projects/ParlaMint/Data/ParlaMint-NO/*.xml"


ns = {"tei" : "http://www.tei-c.org/ns/1.0"}
parser = et.XMLParser(remove_blank_text=True)


In [2]:
for doc in glob(file_paths):
    if "ana" in doc.split("."):
        xml = et.parse(doc, parser)
        #print(doc)
        break

In [4]:
def rewrite_tag(link1, *links):
    "Make link2 a dependant of link 1"
    root = link1.attrib["target"].split()[1]
    
    for link in links:
        link.attrib["ana"] = "ud-syn:parataxis"
        self_ = link.attrib["target"].split()[1]
        link.attrib["target"] = " ".join([root, self_])           
        #link.attrib["target"] = "JULETRE"
    

In [5]:
tot = 0

for linkGrp in xml.findall("//{http://www.tei-c.org/ns/1.0}linkGrp"):
    links = linkGrp.findall("{http://www.tei-c.org/ns/1.0}link")
    
    count = 0
    links_to_change = list()
    
    for link in links:
        if link.attrib["ana"] == "ud-syn:root":
            count += 1
            links_to_change.append(link)
            
    rewrite_tag(links_to_change[0], *links_to_change[1:])
                
    if count > 1:
        #print(count)
        tot += 1
        
        
        
print(tot / len(xml.findall("//{http://www.tei-c.org/ns/1.0}linkGrp")))
            
    

0.04708383961117862


In [6]:
xml.write("test.xml", pretty_print=True, encoding='utf-8')

In [30]:
def process_sent(s):
    id = s.attrib['{http://www.w3.org/XML/1998/namespace}id']
    
    linkGrp = s.find("{http://www.tei-c.org/ns/1.0}linkGrp")
    links = linkGrp.findall("{http://www.tei-c.org/ns/1.0}link")
    
    for link in links:
        if link.attrib["ana"] == "ud-syn:root":
            old_attrib_link =  link.attrib["target"].split()[1]
            new_attrib = f"#{id} {old_attrib_link}" 
            link.attrib["target"] = new_attrib

In [31]:
def parse_doc(doc):
    xml = et.parse(doc, parser)

    sents = xml.xpath("//tei:s", namespaces=ns)
    
    for s in sents:
        process_sent(s)
    
    
    
    xml.write(doc, pretty_print=True, encoding='utf-8')


In [32]:
for doc in glob(file_paths):
    parse_doc(doc)